## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-04-16-57-04 +0000,nypost,Trial begins for suspects in Moscow concert ha...,https://nypost.com/2025/08/04/world-news/trial...
1,2025-08-04-16-52-00 +0000,wsj,Mexico’s Drug Cartels Bedevil Sheinbaum’s Deal...,https://www.wsj.com/world/americas/mexicos-dru...
2,2025-08-04-16-50-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
3,2025-08-04-16-49-26 +0000,nyt,Russian Nuclear Submarine Base Was Damaged in ...,https://www.nytimes.com/2025/08/04/world/europ...
4,2025-08-04-16-49-10 +0000,nyt,Trump’s Demand to Trading Partners: Pledge Mon...,https://www.nytimes.com/2025/08/04/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
17,trump,37
79,new,20
207,texas,8
309,gaza,8
380,deal,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
175,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...,75
163,2025-08-04-04-46-00 +0000,wsj,Democrats in the Texas state House fled the st...,https://www.wsj.com/politics/policy/texas-demo...,72
105,2025-08-04-11-47-24 +0000,nyt,Trump to Appoint New Top Labor Statistics Offi...,https://www.nytimes.com/2025/08/03/us/politics...,70
103,2025-08-04-11-53-00 +0000,wsj,Eric and Donald Trump Jr. to Own Millions of S...,https://www.wsj.com/finance/investing/eric-and...,69
149,2025-08-04-06-47-01 +0000,nypost,Fighter jets scrambled after civilian aircraft...,https://nypost.com/2025/08/04/us-news/fighter-...,69


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
175,75,2025-08-04-00-50-00 +0000,wsj,A chief White House economic adviser said that...,https://www.wsj.com/economy/jobs/trump-seeks-b...
65,45,2025-08-04-14-32-56 +0000,bbc,Watch: Fire breaks out on commuter train conne...,https://www.bbc.com/news/videos/c39dvw08nvdo?a...
48,35,2025-08-04-15-16-00 +0000,wsj,Elon Musk Gets $23.7 Billion Stock Award From ...,https://www.wsj.com/business/autos/tesla-appro...
197,33,2025-08-03-21-35-51 +0000,nypost,"Israel arming anti-Hamas Gaza clans, including...",https://nypost.com/2025/08/03/world-news/israe...
29,32,2025-08-04-15-57-43 +0000,nypost,Ex-Flagstar CEO tipped off client on money-lau...,https://nypost.com/2025/08/04/business/ex-bank...
110,31,2025-08-04-11-07-32 +0000,nypost,Firebrand Rep. Nancy Mace launches campaign fo...,https://nypost.com/2025/08/04/us-news/nancy-ma...
178,29,2025-08-04-00-31-56 +0000,nypost,Russian volcano erupts for first time in centu...,https://nypost.com/2025/08/03/world-news/russi...
188,28,2025-08-03-22-56-15 +0000,nypost,Boeing St. Louis defense workers set to strike...,https://nypost.com/2025/08/03/business/boeing-...
113,28,2025-08-04-11-00-17 +0000,nypost,"Futuristic air-taxi firm Joby expands NYC, glo...",https://nypost.com/2025/08/04/us-news/futurist...
0,27,2025-08-04-16-57-04 +0000,nypost,Trial begins for suspects in Moscow concert ha...,https://nypost.com/2025/08/04/world-news/trial...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
